# 熱中症患者数予測
#### 気象データから熱中症患者数を予測する回帰モデルを作成する

## Azure ML Service用の準備

In [55]:
import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.17


In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/config.json
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FH9U5C7CP to authenticate.
Interactive authentication successfully completed.
mlservice4DLLAB	eastus	rsg4DLLAB	eastus


In [3]:
experiment_name = 'Heatstroke_patient_prediction2'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

import pandas as pd

project_folder = './sample_projects/Heatstroke_patient_prediction2'
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = exp.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Experiment Name,Heatstroke_patient_prediction2
Location,eastus
Project Directory,./sample_projects/Heatstroke_patient_prediction2
Resource Group,rsg4DLLAB
SDK version,1.0.17
Subscription ID,66de4f5c-3bb4-4c23-be1a-ad0629241a58
Workspace Name,mlservice4DLLAB


## トレーニング用データをストレージにアップロード

In [4]:
import pandas as pd
df = pd.read_csv('./data/Heatstroke_patient_prediction_train_data.csv')

In [5]:
df.head(10)

,年月日,搬送人員（計）,最高気温(℃),平均気温(℃),最低気温(℃),日照時間(時間),平均風速(m/s),平均雲量(10分比),平均湿度(％),降水量の合計(mm),...,m前日最高気温との差,m前日平均気温との差,m前日最低気温との差,m最高気温移動平均(5日間),m平均気温移動平均(5日間),m体感温度移動平均(5日間),m不快指数移動平均(5日間),m前日の搬送人数,m搬送人数移動平均(5日間),年
0,7/2/2008 12:00:00 AM,4,26.9,23.7,21.1,8.3,3.1,9.5,70,0.0,...,1.4,1.9,2.5,24.82,21.90,19.750477,69.154300,0,0.000000,2008
1,7/3/2008 12:00:00 AM,1,27.3,24.3,22.1,1.4,5.0,7.8,75,0.0,...,0.4,0.6,1.0,25.14,22.32,20.025609,69.813896,4,2.000000,2008
2,7/4/2008 12:00:00 AM,14,31.6,26.5,23.1,8.7,3.7,7.8,77,16.0,...,4.3,2.2,1.0,25.38,22.50,20.171603,70.182006,1,1.666667,2008
3,7/5/2008 12:00:00 AM,26,31.4,27.5,24.9,8.0,2.3,7.8,72,0.0,...,-0.2,1.0,1.8,27.20,23.60,22.279985,71.768756,14,4.750000,2008
4,7/6/2008 12:00:00 AM,15,30.7,27.0,24.7,4.2,2.5,10.0,76,0.0,...,-0.7,-0.5,-0.2,28.54,24.76,23.812901,73.535204,26,9.000000,2008
5,7/7/2008 12:00:00 AM,1,26.9,25.1,24.0,0.0,2.1,10.0,81,15.0,...,-3.8,-1.9,-0.7,29.58,25.80,25.127460,75.232920,15,12.000000,2008
6,7/8/2008 12:00:00 AM,1,26.5,24.1,22.8,0.3,3.0,10.0,80,1.0,...,-0.4,-1.0,-1.2,29.58,26.08,25.375582,75.885838,1,11.400000,2008
7,7/9/2008 12:00:00 AM,2,26.1,24.3,22.7,1.2,2.5,10.0,68,0.0,...,-0.4,0.2,-0.1,29.42,26.04,25.393262,75.919328,1,11.400000,2008
8,7/10/2008 12:00:00 AM,2,27.8,24.6,22.1,2.5,2.3,9.0,70,0.0,...,1.7,0.3,-0.6,28.32,25.60,24.053976,75.051890,2,9.000000,2008
9,7/11/2008 12:00:00 AM,6,29.4,26.0,23.5,5.5,3.1,8.8,71,0.0,...,1.6,1.4,1.4,27.60,25.02,23.196735,74.128450,2,4.200000,2008


In [6]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob mlservice4dlla9175195115 azureml-blobstore-49463792-29cf-4242-ad82-74ad34ac9d6e


In [7]:
ds.upload_files(['./data/Heatstroke_patient_prediction_train_data.csv'],overwrite=True)

Uploading ./data/Heatstroke_patient_prediction_train_data.csv
Uploaded ./data/Heatstroke_patient_prediction_train_data.csv, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_workspaceblobstore

## トレーニング実行用の ML Compute 作成

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D4_V2', #'Standard_NC6'
                                                           min_nodes=0,
                                                           max_nodes=8)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


## Automated ML 機能を使って予測モデル作成

In [9]:
import os
os.makedirs(project_folder, exist_ok=True)

### モデル学習用スクリプト作成

In [10]:
%%writefile $project_folder/get_data.py

import numpy as np
from azure.storage.blob import BlockBlobService
import pandas as pd
import os.path

def get_data():
    file_name = "Heatstroke_patient_prediction_train_data.csv"
    if not os.path.exists('./' + file_name) :
        account_name='<ストレージアカウント名>'
        account_key='<ストレージアカウントキー>'
        container_name='<コンテナ名>'
        blob_name=file_name
        
        service = BlockBlobService(account_name=account_name, account_key=account_key)
        service.get_blob_to_path(container_name, blob_name, file_name)
    
    df = pd.read_csv('./' + file_name)
    X_train = df.drop(columns=["年月日","年","月","搬送人員（計）"],axis=1)
    y_df = df["搬送人員（計）"]
    y_train = y_df.values

    return { "X" : X_train, "y" : y_train }

Overwriting ./sample_projects/Heatstroke_patient_prediction2/get_data.py


### モデル学習実行環境定義

In [11]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = cluster.name

# enable Docker 
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy','pandas'], 
    pip_packages=['azureml-sdk[automl]', 'azure-storage'])

### automated ML 実行

In [12]:
from azureml.train.automl import AutoMLConfig
import logging

automated_ml_config = AutoMLConfig(task = 'regression',
                             primary_metric = 'normalized_mean_absolute_error',
                             iteration_timeout_minutes = 10,
                             iterations = 30,
                             preprocess = True,
                             verbosity = logging.INFO,
                             experiment_exit_score = 0,
                             path = project_folder,
                             data_script = project_folder + "/get_data.py",
                             run_configuration=run_config,
                             debug_log = 'automated_ml_errors.log',
                             n_cross_validations = 3,
                             max_concurrent_iterations = 4)

In [13]:
run_automl = exp.submit(automated_ml_config, show_output = False)
run_automl

Experiment,Id,Type,Status,Details Page,Docs Page
Heatstroke_patient_prediction2,AutoML_668d9998-a330-4393-88f0-f9fe5bd17071,automl,Preparing,Link to Azure Portal,Link to Documentation


In [14]:
from azureml.widgets import RunDetails
RunDetails(run_automl).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
%%time
# Shows output of the run on stdout.
run_automl.wait_for_completion(show_output=True)


********************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
SAMPLING %: Percent of the training data to sample.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
********************************************************************************************************************

 ITERATION   PIPELINE                                       SAMPLING %  DURATION      METRIC      BEST
         3   StandardScalerWrapper ExtremeRandomTrees       100.0000    0:01:20       0.0276    0.0276
         2   StandardScalerWrapper RandomForest             100.0000    0:00:32       0.0355    0.0276
         1   StandardScalerWrapper ElasticNet               100.0000    0:00:56       0.0388    0.0276
         0   StandardScalerWrapper ElasticN

{'runId': 'AutoML_668d9998-a330-4393-88f0-f9fe5bd17071',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-04-23T05:06:56.14988Z',
 'endTimeUtc': '2019-04-23T05:16:39.199821Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'MaxTimeSeconds': '600',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpucluster',
  'RawAMLSettingsString': "{'name': 'Heatstroke_patient_prediction2', 'path': './sample_projects/Heatstroke_patient_prediction2', 'subscription_id': '66de4f5c-3bb4-4c23-be1a-ad0629241a58', 'resource_group': 'rsg4DLLAB', 'workspace_name': 'mlservice4DLLAB', 'iterations': 30, 'primary_metric': 'normalized_mean_absolute_error', 'data_script': './sample_projects/Heatstroke_patient_prediction2/get_data.py', 'compute_target': 'cpucluster', 'task_type': 'regression', 'validation_size': 0.0, 'n_cross_validat

In [16]:
children = list(run_automl.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
explained_variance,0.80,0.79,0.75,0.85,0.77,0.79,0.83,0.87,0.79,0.76,...,0.86,0.80,0.79,0.79,0.77,0.78,0.82,0.85,0.87,0.87
mean_absolute_error,11.37,11.76,10.76,8.35,10.08,9.88,10.82,7.80,11.61,10.54,...,8.01,11.39,11.70,11.70,10.50,11.76,9.20,8.45,7.98,7.86
median_absolute_error,7.88,8.17,4.95,3.92,4.20,3.73,7.47,3.34,7.80,4.13,...,3.58,7.99,8.01,8.14,4.65,8.04,3.98,3.81,3.62,3.74
normalized_mean_absolute_error,0.04,0.04,0.04,0.03,0.03,0.03,0.04,0.03,0.04,0.03,...,0.03,0.04,0.04,0.04,0.03,0.04,0.03,0.03,0.03,0.03
normalized_median_absolute_error,0.03,0.03,0.02,0.01,0.01,0.01,0.02,0.01,0.03,0.01,...,0.01,0.03,0.03,0.03,0.02,0.03,0.01,0.01,0.01,0.01
normalized_root_mean_squared_error,0.06,0.06,0.07,0.05,0.06,0.06,0.05,0.05,0.06,0.06,...,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.05,0.05
normalized_root_mean_squared_log_error,nan,nan,0.13,0.11,0.12,0.11,nan,0.10,nan,0.12,...,0.10,nan,nan,nan,0.13,nan,0.11,0.11,0.10,nan
r2_score,0.80,0.79,0.75,0.85,0.77,0.79,0.83,0.87,0.79,0.76,...,0.86,0.80,0.79,0.79,0.77,0.78,0.82,0.85,0.87,0.87
root_mean_squared_error,17.65,18.27,20.01,15.29,18.89,18.37,16.52,14.28,18.22,19.46,...,14.69,17.68,18.22,18.21,19.32,18.59,16.95,15.43,14.44,14.23
root_mean_squared_log_error,nan,nan,0.76,0.61,0.67,0.65,nan,0.58,nan,0.70,...,0.58,nan,nan,nan,0.72,nan,0.64,0.62,0.59,nan


## モデル登録

In [17]:
best_run, fitted_model = run_automl.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Heatstroke_patient_prediction2,
Id: AutoML_668d9998-a330-4393-88f0-f9fe5bd17071_7,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_imp...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])


In [18]:
description = 'Heatstroke patient prediction Model2'
tags = None
model = run_automl.register_model(description=description, tags=tags)
print(run_automl.model_id) # Use this id to deploy the model as a web service in Azure

Registering model AutoML668d9998abest
AutoML668d9998abest
